In [52]:
# pip install qrcode[pil]

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [26]:
# pip install pycryptodome

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for pycryptodome from https://files.pythonhosted.org/packages/1f/90/d131c0eb643290230dfa4108b7c2d135122d88b714ad241d77beb4782a76/pycryptodome-3.20.0-cp35-abi3-win_amd64.whl.metadata
  Using cached pycryptodome-3.20.0-cp35-abi3-win_amd64.whl.metadata (3.4 kB)
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
    --------------------------------------- 0.0/1.8 MB 1.3 MB/s eta 0:00:02
    --------------------------------------- 0.0/1.8 MB 388.9 kB/s eta 0:00:05
   -- ------------------------------------- 0.1/1.8 MB 930.9 kB/s eta 0:00:02
   --- ------------------------------------ 0.1/1.8 MB 847.9 kB/s eta 0:00:02
   ----- ---------------------------------- 0.2/1.8 MB 1.3 MB/s eta 0:00:02
   ----- ---------------------------------- 0.3/1.8 MB 983.0 kB/s eta 0:00:02
   ----- ---------------------------------- 0.3/1.8 MB 983.0 kB/s eta 0:00:02
   -------- -----

In [14]:
import numpy as np
import pandas as pd 
import qrcode
import matplotlib.pyplot as plt
import plotly.express as px
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.preprocessing import scale 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score,roc_curve
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import os

from warnings import filterwarnings



filterwarnings('ignore')

In [15]:
df = pd.read_csv('Sleep_health_and_lifestyle_dataset.csv', sep = ',')

**Dataset Genel Bakış:**

*Uyku Sağlığı ve Yaşam Tarzı Veri Seti*, 400 satır ve 13 sütundan oluşmaktadır. Geniş bir değişken yelpazesini kapsayan bu veri seti, uyku ve günlük alışkanlıklarla ilgili detayları içerir. Cinsiyet, yaş, meslek, uyku süresi, uyku kalitesi, fiziksel aktivite seviyesi, stres düzeyleri, BMI kategorisi, kan basıncı, kalp atış hızı, günlük adımlar ve uyku bozukluklarının varlığı veya yokluğu gibi bilgileri içerir.

**Dataset'in Temel Özellikleri:**

1. **Kapsamlı Uyku Metrikleri:** Uyku süresini, kalitesini ve uyku düzenini etkileyen faktörleri keşfedin.
2. **Yaşam Tarzı Faktörleri:** Fiziksel aktivite seviyelerini, stres düzeylerini ve BMI kategorilerini analiz edin.
3. **Kardiyovasküler Sağlık:** Kan basıncı ve kalp atış hızı ölçümlerini inceleyin.
4. **Uyku Bozukluğu Analizi:** İnsomnia ve Sleep Apnea gibi uyku bozukluklarının varlığını belirleyin.

**Dataset Sütunları:**

1. **Person ID:** Her birey için bir tanımlayıcı.
2. **Gender:** Bireyin cinsiyeti (Erkek/Kadın).
3. **Age:** Bireyin yaşı (yıl olarak).
4. **Occupation:** Bireyin mesleği veya işi.
5. **Sleep Duration (saat):** Bireyin günde kaç saat uyuduğu.
6. **Quality of Sleep (ölçek: 1-10):** Uyku kalitesinin subjektif bir değerlendirmesi, 1 ila 10 arasında.
7. **Physical Activity Level (dakika/gün):** Bireyin günlük fiziksel aktivite süresi.
8. **Stress Level (ölçek: 1-10):** Birey tarafından deneyimlenen stres düzeyinin subjektif bir değerlendirmesi, 1 ila 10 arasında.
9. **BMI Category:** Bireyin BMI kategorisi (Örneğin, Zayıf, Normal, Kilolu).
10. **Blood Pressure (systolic/diastolic):** Bireyin kan basıncı ölçümü, sistolik basınç üzeri diyastolik basınç olarak gösterilmiştir.
11. **Heart Rate (bpm):** Bireyin dakikadaki atım hızı.
12. **Daily Steps:** Bireyin günde attığı adım sayısı.
13. **Sleep Disorder:** Bireyde uyku bozukluğunun varlığı veya yokluğu (Hiçbiri, Insomnia, Sleep Apnea).

**Uyku Bozukluğu Sütunu Hakkında Detaylar:**

- **None:** Bireyde herhangi belirgin bir uyku bozukluğu bulunmamaktadır.
- **Insomnia:** Birey uykuya dalmakta veya uykusunu sürdürmekte zorlanır, bu da yetersiz veya kalitesiz uykuya yol açar.
- **Sleep Apnea:** Birey uykusunda nefes alma duraklamaları yaşar, bu da bozulmuş uyku düzenine ve potansiyel sağlık risklerine neden olabilir.



In [16]:
# Nitelik isimlerini düzenleme fonksiyonu
def düzenle_nitelik_isimleri(df):
    for column in df.columns:
        # İki kelimeden oluşan sütunları "_" ile birleştir
        if ' ' in column:
            new_column = column.replace(' ', '_')
            df = df.rename(columns={column: new_column})
    return df
df = düzenle_nitelik_isimleri(df)


In [17]:
from sklearn.preprocessing import LabelEncoder

In [18]:
label_encoder=LabelEncoder()

In [19]:
cat_cols=['Gender','Occupation','BMI_Category']
for col in cat_cols:
    df[col] = label_encoder.fit_transform(df[col])

In [20]:
 # Split the 'Blood Pressure' column into two columns
df[['Systolic_BP', 'Diastolic_BP']] = df['Blood_Pressure'].str.split('/', expand=True)
    
# Convert the new columns to numeric type
df[['Systolic_BP', 'Diastolic_BP']] = df[['Systolic_BP', 'Diastolic_BP']].apply(pd.to_numeric)
    
# Drop the original 'Blood Pressure' column
del(df['Blood_Pressure'])   # Hesapları yanıltmaması için sildim

In [21]:
def encode(x):
    if x == "Sleep Apnea":
        return 1
    elif x == "Insomnia":
        return 2
    else:
        return 0

df["Sleep_Disorder"] = df["Sleep_Disorder"].apply(lambda x: encode(x) if pd.notna(x) else 0)


In [22]:
df.Sleep_Disorder.value_counts()


0    219
1     78
2     77
Name: Sleep_Disorder, dtype: int64

In [23]:
del(df['Person_ID'])   # Hesapları yanıltmaması için sildim

In [24]:
df.head(10)

,Gender,Age,Occupation,Sleep_Duration,Quality_of_Sleep,Physical_Activity_Level,Stress_Level,BMI_Category,Heart_Rate,Daily_Steps,Sleep_Disorder,Systolic_BP,Diastolic_BP
0,1,27,9,6.1,6,42,6,3,77,4200,0,126,83
1,1,28,1,6.2,6,60,8,0,75,10000,0,125,80
2,1,28,1,6.2,6,60,8,0,75,10000,0,125,80
3,1,28,6,5.9,4,30,8,2,85,3000,1,140,90
4,1,28,6,5.9,4,30,8,2,85,3000,1,140,90
5,1,28,9,5.9,4,30,8,2,85,3000,2,140,90
6,1,29,10,6.3,6,40,7,2,82,3500,2,140,90
7,1,29,1,7.8,7,75,6,0,70,8000,0,120,80
8,1,29,1,7.8,7,75,6,0,70,8000,0,120,80
9,1,29,1,7.8,7,75,6,0,70,8000,0,120,80


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Gender                   374 non-null    int32  
 1   Age                      374 non-null    int64  
 2   Occupation               374 non-null    int32  
 3   Sleep_Duration           374 non-null    float64
 4   Quality_of_Sleep         374 non-null    int64  
 5   Physical_Activity_Level  374 non-null    int64  
 6   Stress_Level             374 non-null    int64  
 7   BMI_Category             374 non-null    int32  
 8   Heart_Rate               374 non-null    int64  
 9   Daily_Steps              374 non-null    int64  
 10  Sleep_Disorder           374 non-null    int64  
 11  Systolic_BP              374 non-null    int64  
 12  Diastolic_BP             374 non-null    int64  
dtypes: float64(1), int32(3), int64(9)
memory usage: 33.7 KB


### Anlamlı QR oluşturma Çalışmaları

In [27]:
import qrcode
import os
import pandas as pd

def veriyi_qr_koda_cevir(veri, dosya_adi):
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    qr.add_data(veri)
    qr.make(fit=True)

    img = qr.make_image(fill_color="black", back_color="white")
    img.save(dosya_adi)

if __name__ == "__main__":


    # Kaydetmek istediğiniz ana dosya yolu
    ana_dosya_yolu = r"C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir"

    # Her bir hedef klasör için QR kodlarını oluştur
    for index, row in df.iterrows():
        # Sadece numerik nitelikleri al
        numerik_veri = row.drop('Sleep_Disorder').to_numpy()
        
        # Nitelikleri ve sonucu bir araya getir
        veri = ','.join(map(str, numerik_veri)) + ':' + str(row['Sleep_Disorder'])
        
        # Hedef klasör yolunu belirleyin
        klasor_yolu = os.path.join(ana_dosya_yolu, f"hedef_deger_{row['Sleep_Disorder']}")
        
        # QR kodlarını kaydetmek için klasörü oluştur
        if not os.path.exists(klasor_yolu):
            os.makedirs(klasor_yolu)

        # QR kodunun dosya yolunu oluştur
        qr_dosya_adi = os.path.join(klasor_yolu, f"qr_kod_{index}.png")
        
        # Veriyi QR koda dönüştür ve kaydet
        veriyi_qr_koda_cevir(veri, qr_dosya_adi)
        print(f"{qr_dosya_adi} dosyası başarıyla kaydedildi.")


C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_0.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_1.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_2.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_1.0\qr_kod_3.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_1.0\qr_kod_4.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_2.0\qr_kod_5.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_2.0\qr_kod_6.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_7.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\

C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_87.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_88.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_89.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_90.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_91.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_92.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_1.0\qr_kod_93.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_2.0\qr_kod_94.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\

C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_163.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_164.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_2.0\qr_kod_165.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_166.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_167.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_168.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_169.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_170.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\

C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_2.0\qr_kod_247.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_248.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_249.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_2.0\qr_kod_250.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_2.0\qr_kod_251.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_2.0\qr_kod_252.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_2.0\qr_kod_253.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_2.0\qr_kod_254.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\

C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_334.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_335.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_336.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_337.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_338.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_1.0\qr_kod_339.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_1.0\qr_kod_340.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_341.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\

### Bu kod abuk subuk bi çıktı verdi kullanılabilir.

In [67]:
import qrcode
import os
import pandas as pd

def veriyi_qr_koda_cevir(veri, dosya_adi):
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    qr.add_data(veri)
    qr.make(fit=True)

    img = qr.make_image(fill_color="black", back_color="white")
    img.save(dosya_adi)

if __name__ == "__main__":
    # Önceden yüklenmiş veri seti DataFrame'i
    # Veri setinizi yükleyin veya oluşturun

    # Kaydetmek istediğiniz ana dosya yolu
    ana_dosya_yolu = r"C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir"

    # Her bir hedef klasör için QR kodlarını oluştur
    for index, row in df.iterrows():
        # Hedef değişken değerini al
        hedef_deger = row['Sleep_Disorder']
        
        # Hedef değişken değerine göre uygun klasör yolu belirleyin
        if hedef_deger == 0:
            klasor_yolu = os.path.join(ana_dosya_yolu, "hasta_degil0")
        elif hedef_deger == 1:
            klasor_yolu = os.path.join(ana_dosya_yolu, "Sleep_Apnea1")
        elif hedef_deger == 2:
            klasor_yolu = os.path.join(ana_dosya_yolu, "Insomnia2")
        else:
            continue  # Hedef değişken başka bir değere sahipse işlemi atla

        # QR kodlarını kaydetmek için klasörü oluştur
        if not os.path.exists(klasor_yolu):
            os.makedirs(klasor_yolu)

        # Veriyi al
        veri = str(row.values)
        # QR kodunun dosya yolunu oluştur
        qr_dosya_adi = os.path.join(klasor_yolu, f"qr_kod_{index}.png")
        # Veriyi QR koda dönüştür ve kaydet
        veriyi_qr_koda_cevir(veri, qr_dosya_adi)
        print(f"{qr_dosya_adi} dosyası başarıyla kaydedildi.")


C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hasta_degil0\qr_kod_0.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hasta_degil0\qr_kod_1.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hasta_degil0\qr_kod_2.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\Sleep_Apnea1\qr_kod_3.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\Sleep_Apnea1\qr_kod_4.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\Insomnia2\qr_kod_5.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\Insomnia2\qr_kod_6.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hasta_degil0\qr_kod_7.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hasta_degil0\qr_ko

C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\Sleep_Apnea1\qr_kod_81.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hasta_degil0\qr_kod_82.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hasta_degil0\qr_kod_83.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hasta_degil0\qr_kod_84.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hasta_degil0\qr_kod_85.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hasta_degil0\qr_kod_86.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hasta_degil0\qr_kod_87.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hasta_degil0\qr_kod_88.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hast

C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\Insomnia2\qr_kod_165.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hasta_degil0\qr_kod_166.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hasta_degil0\qr_kod_167.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hasta_degil0\qr_kod_168.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hasta_degil0\qr_kod_169.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hasta_degil0\qr_kod_170.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hasta_degil0\qr_kod_171.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hasta_degil0\qr_kod_172.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir

C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\Insomnia2\qr_kod_246.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\Insomnia2\qr_kod_247.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hasta_degil0\qr_kod_248.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hasta_degil0\qr_kod_249.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\Insomnia2\qr_kod_250.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\Insomnia2\qr_kod_251.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\Insomnia2\qr_kod_252.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\Insomnia2\qr_kod_253.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\Insomnia2\qr_k

C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hasta_degil0\qr_kod_326.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hasta_degil0\qr_kod_327.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hasta_degil0\qr_kod_328.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hasta_degil0\qr_kod_329.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hasta_degil0\qr_kod_330.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hasta_degil0\qr_kod_331.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hasta_degil0\qr_kod_332.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hasta_degil0\qr_kod_333.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflan

In [66]:
import qrcode
import os
import pandas as pd

def veriyi_qr_koda_cevir(veri, dosya_adi):
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    qr.add_data(veri)
    qr.make(fit=True)

    img = qr.make_image(fill_color="black", back_color="white")
    img.save(dosya_adi)

if __name__ == "__main__":
    # Önceden yüklenmiş veri seti DataFrame'i
    # Veri setinizi yükleyin veya oluşturun

    # Kaydetmek istediğiniz ana dosya yolu
    ana_dosya_yolu = r"C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir"

    # Her bir hedef klasör için QR kodlarını oluştur
    for index, row in df.iterrows():
        # Sadece numerik nitelikleri al
        numerik_veri = row.to_numpy()
        
        # Hedef klasör yolunu belirleyin
        klasor_yolu = os.path.join(ana_dosya_yolu, f"hedef_deger_{row['Sleep_Disorder']}")
        
        # QR kodlarını kaydetmek için klasörü oluştur
        if not os.path.exists(klasor_yolu):
            os.makedirs(klasor_yolu)

        # Veriyi QR koduna dönüştür
        veri = ','.join(map(str, numerik_veri))
        
        # QR kodunun dosya yolunu oluştur
        qr_dosya_adi = os.path.join(klasor_yolu, f"qr_kod_{index}.png")
        
        # Veriyi QR koda dönüştür ve kaydet
        veriyi_qr_koda_cevir(veri, qr_dosya_adi)
        print(f"{qr_dosya_adi} dosyası başarıyla kaydedildi.")


C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_0.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_1.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_2.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_1.0\qr_kod_3.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_1.0\qr_kod_4.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_2.0\qr_kod_5.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_2.0\qr_kod_6.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_7.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\

C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_85.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_86.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_87.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_88.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_89.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_90.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_91.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_92.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\

C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_169.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_170.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_171.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_172.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_173.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_174.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_175.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_176.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\

C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_2.0\qr_kod_251.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_2.0\qr_kod_252.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_2.0\qr_kod_253.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_2.0\qr_kod_254.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_2.0\qr_kod_255.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_2.0\qr_kod_256.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_2.0\qr_kod_257.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_2.0\qr_kod_258.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\

C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_329.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_330.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_331.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_332.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_333.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_334.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_335.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\siniflandir\hedef_deger_0.0\qr_kod_336.png dosyası başarıyla kaydedildi.
C:\Users\Asus\OneDrive\Masaüstü\

# Verileri Şifrele

### Deneme 1

In [60]:
from cryptography.fernet import Fernet

In [90]:
"""# Veriyi bir metin dizesine dönüştür
df_metin = df.to_csv(index=False)"""

'# Veriyi bir metin dizesine dönüştür\ndf_metin = df.to_csv(index=False)'

In [91]:
"""# AES şifreleme anahtarı oluştur
anahtar = Fernet.generate_key()"""

'# AES şifreleme anahtarı oluştur\nanahtar = Fernet.generate_key()'

In [92]:
"""# AES şifreleme nesnesi oluştur
cipher_suite = Fernet(anahtar)"""

'# AES şifreleme nesnesi oluştur\ncipher_suite = Fernet(anahtar)'

In [93]:
"""# Veriyi şifrele
sifreli_veri = cipher_suite.encrypt(df_metin.encode())"""

'# Veriyi şifrele\nsifreli_veri = cipher_suite.encrypt(df_metin.encode())'

In [94]:
"""# Şifreli veriyi bir dosyaya yaz
with open("sifreli_veri.bin", "wb") as dosya:
    dosya.write(sifreli_veri)"""

'# Şifreli veriyi bir dosyaya yaz\nwith open("sifreli_veri.bin", "wb") as dosya:\n    dosya.write(sifreli_veri)'

## Başarısız istenmeyen durum

### Deneme 2

In [95]:
"""from Crypto.Cipher import AES
# AES şifreleme anahtarı oluştur
anahtar = Fernet.generate_key()
print(anahtar)
# AES şifreleme nesnesi oluştur
cipher_suite = Fernet(anahtar)
# Her bir veriyi parçalı parçalı şifrele ve QR koda dönüştür
for i, row in df.iterrows():
    veri = str(row.values)  # Veriyi al
    sifreli_veri = cipher_suite.encrypt(veri.encode())  # Veriyi şifrele
    qr = qrcode.make(sifreli_veri.decode())  # Şifreli veriyi QR koda dönüştür
    qr_dosya_adi = os.path.join(r"C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\sifreli_Veri", f"qr_kod_{i}.png")  # QR kodunun dosya adı
    qr.save(qr_dosya_adi)  # QR kodunu belirtilen dosya yoluna kaydet
    print(f"{qr_dosya_adi} dosyası oluşturuldu.")
print("Şifreleme Anahtarı:", anahtar.decode())



"""

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 671-672: truncated \UXXXXXXXX escape (3950400942.py, line 19)

In [96]:
from Crypto.Cipher import AES

In [97]:
# AES şifreleme anahtarı oluştur
anahtar = Fernet.generate_key()
print(anahtar)

Şifreleme Anahtarı: 1234567890123456


In [98]:
cipher_suite = AES.new(anahtar, AES.MODE_ECB)


In [104]:
# Her bir veriyi parçalı parçalı şifrele ve QR koda dönüştür
for i, row in df.iterrows():
    veri = str(row.values)  # Veriyi al
    sifreli_veri = cipher_suite.encrypt(veri.encode())  # Veriyi şifrele
    qr = qrcode.make(sifreli_veri.decode())  # Şifreli veriyi QR koda dönüştür
    qr_dosya_adi = os.path.join(r"C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\sifreli_Veri", f"qr_kod_{i}.png")  # QR kodunun dosya adı
    qr.save(qr_dosya_adi)  # QR kodunu belirtilen dosya yoluna kaydet
    print(f"{qr_dosya_adi} dosyası oluşturuldu.")


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc7 in position 0: invalid continuation byte

In [74]:
print("Şifreleme Anahtarı:", anahtar.decode())


Şifreleme Anahtarı: QumPgenMwzD__ooqsO1ovNneuRyrbrHrmH_m879ByW4


In [86]:
"""# AES şifreleme anahtarı
anahtar = b'AoI-d8YzA3xiGcM788AxTcoYmpPAY8c6Lpn8ytsst8s-c='  # Şifreleme sırasında kullanılan anahtarı buraya girin

# AES şifreleme nesnesi oluştur
cipher_suite = Fernet(anahtar)

# Şifrelenmiş verileri saklayacak bir liste oluştur
sifreli_veriler = []

# Şifreli verileri oku
for dosya_adi in os.listdir(r"C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\sifreli_Veri"):  # Şifreli verilerin bulunduğu klasörün yolu
    with open(os.path.join(r"C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\sifreli_Veri", dosya_adi), "rb") as dosya:
        sifreli_veri = dosya.read()
        sifreli_veriler.append(sifreli_veri)

# Şifrelenmiş verileri çöz
cozulmus_veriler = []
for sifreli_veri in sifreli_veriler:
    cozulmus_veri = cipher_suite.decrypt(sifreli_veri)
    cozulmus_veriler.append(cozulmus_veri.decode())

# Çözülen verileri birleştirerek DataFrame'e dönüştür
veri_seti_cozulmus = pd.DataFrame([veri.split(",") for veri in cozulmus_veriler])

# Veri setini görüntüle
print(veri_seti_cozulmus)"""

SyntaxError: incomplete input (1655350369.py, line 26)

In [77]:
# Anahtar ve şifreleme nesnesini tanımla
anahtar = b'QumPgenMwzD__ooqsO1ovNneuRyrbrHrmH_m879ByW4'
cipher_suite = AES.new(anahtar, AES.MODE_ECB)

ValueError: Incorrect AES key length (43 bytes)

In [77]:
# QR kodlarından veriyi çöz ve kaydet
sifreli_veriler_dizini = r"C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\sifreli_Veri"
cozulmus_veriler_dizini = r"C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\cozulmus_Veri"

In [78]:
# Çözülmüş verilerin dizinini oluştur
if not os.path.exists(cozulmus_veriler_dizini):
    os.makedirs(cozulmus_veriler_dizini)

In [85]:
for dosya_adi in os.listdir(sifreli_veriler_dizini):
    if dosya_adi.endswith(".png"):
        qr_dosya_yolu = os.path.join(sifreli_veriler_dizini, dosya_adi)
        qr_img = qrcode.image.imread(qr_dosya_yolu)  # QR kodunu oku
        qr_verisi = qrcode.decoder.decode(qr_img)  # QR kodunun verisini çöz

        # QR kodunu çöz
        sifreli_veri = qr_verisi[0].data.decode()
        cozulmus_veri = cipher_suite.decrypt(sifreli_veri.encode()).decode()

        # Dosyaya yaz
        dosya_ad = dosya_adi.split('.')[0]
        dosya_yolu = os.path.join(cozulmus_veriler_dizini, f"{dosya_ad}.txt")
        with open(dosya_yolu, 'w') as f:
            f.write(cozulmus_veri)

        print(f"{dosya_yolu} dosyası oluşturuldu.")

print("Tüm veriler çözüldü ve kaydedildi.")


AttributeError: module 'qrcode.image' has no attribute 'imread'

In [ ]:
"""# Şifreli verileri oku
for dosya_adi in os.listdir(r"C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\sifreli_Veri"):  # Şifreli verilerin bulunduğu klasörün yolu
    with open(os.path.join(r"C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\sifreli_Veri", dosya_adi), "rb") as dosya:
        sifreli_veri = dosya.read()
        sifreli_veriler.append(sifreli_veri)
"""

In [ ]:
"""# Şifrelenmiş verileri çöz
cozulmus_veriler = []
for sifreli_veri in sifreli_veriler:
    cozulmus_veri = cipher_suite.decrypt(sifreli_veri)
    cozulmus_veriler.append(cozulmus_veri.decode())"""

In [ ]:
"""# Çözülen verileri birleştirerek DataFrame'e dönüştür
veri_seti_cozulmus = pd.DataFrame([veri.split(",") for veri in cozulmus_veriler])"""

In [ ]:
"""# Veri setini görüntüle
print(veri_seti_cozulmus)"""

In [ ]:
"""def veriyi_qr_koda_cevir(veri, dosya_adi):
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    qr.add_data(veri)
    qr.make(fit=True)

    img = qr.make_image(fill_color="black", back_color="white")
    img.save(dosya_adi)

if __name__ == "__main__":
    # Önceden yüklenmiş veri seti DataFrame'i

    # Kaydetmek istediğiniz dosya yolunu belirleyin
    dosya_yolu = r"C:\Users\Asus\OneDrive\Masaüstü\hüseyin\bitirme\qr_kodlar"

    # QR kodlarını kaydetmek için klasörü oluştur
    if not os.path.exists(dosya_yolu):
        os.makedirs(dosya_yolu)

    # Her bir satırı temsil eden QR kodları oluştur
    for i, row in df.iterrows():
        veri = str(row.values)  # Veriyi al
        qr_dosya_adi = os.path.join(dosya_yolu, f"qr_kod_{i}.png")  # QR kodunun dosya yolu
        veriyi_qr_koda_cevir(veri, qr_dosya_adi)
        print(f"{qr_dosya_adi} dosyası başarıyla kaydedildi.")
"""